Student Name: Huaiying Shao

UOW ID: 7910356

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sklearn dependencies
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

# Tensorflow dependencies
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.models import Model

## Load dataset, explore data

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# read in dataset
path = "/content/drive/MyDrive/dataset/secondary_data.csv"
data = pd.read_csv(path, sep=';')
# view the first five data
data.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


In [ ]:
# view the data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-root             9531 non-null   object 
 12  stem-surface          22945 non-null  object 
 13  stem-color            61069 non-null  object 
 14  veil-type             3177 non-null   object 
 15  veil-color         

Data preprocessing

In [ ]:
# label encode all categorical columns
cat_columns = data.select_dtypes('object').columns

for column in data.columns:
    data[column] = LabelEncoder().fit_transform(data[column])

data.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,1,1481,6,2,6,0,2,3,10,1577,...,4,7,11,0,4,1,2,7,0,3
1,1,1614,6,2,6,0,2,3,10,1681,...,4,7,11,0,4,1,2,7,0,2
2,1,1362,6,2,6,0,2,3,10,1662,...,4,7,11,0,4,1,2,7,0,3
3,1,1372,2,3,1,0,2,3,10,1463,...,4,7,11,0,4,1,5,7,0,3
4,1,1419,6,3,6,0,2,3,10,1537,...,4,7,11,0,4,1,5,7,0,3


Standard Scaling all columns

In [ ]:
all_columns = set(data.columns)

# Exclude the target column
all_columns.remove('season')

for column in all_columns:
    data[column] = StandardScaler().fit_transform(data[column].values.reshape(-1, 1))

data.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0.895591,2.036990,0.915714,-1.287113,-0.045582,-0.458029,-0.47925,1.113142,0.835268,3.217202,...,-0.432597,0.151892,0.798799,-4.268749,-2.011264,1.738751,0.136786,0.308365,-0.492721,3
1,0.895591,2.348960,0.915714,-1.287113,-0.045582,-0.458029,-0.47925,1.113142,0.835268,3.542198,...,-0.432597,0.151892,0.798799,-4.268749,-2.011264,1.738751,0.136786,0.308365,-0.492721,2
2,0.895591,1.757858,0.915714,-1.287113,-0.045582,-0.458029,-0.47925,1.113142,0.835268,3.482824,...,-0.432597,0.151892,0.798799,-4.268749,-2.011264,1.738751,0.136786,0.308365,-0.492721,3
3,0.895591,1.781315,-0.954657,-1.016635,-1.622346,-0.458029,-0.47925,1.113142,0.835268,2.860957,...,-0.432597,0.151892,0.798799,-4.268749,-2.011264,1.738751,1.675068,0.308365,-0.492721,3
4,0.895591,1.891560,0.915714,-1.016635,-0.045582,-0.458029,-0.47925,1.113142,0.835268,3.092204,...,-0.432597,0.151892,0.798799,-4.268749,-2.011264,1.738751,1.675068,0.308365,-0.492721,3


## Build the ANN

split the dataset into train and test set

In [ ]:
features = data[all_columns]
targets = data['season']

# Split the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(features, targets, test_size=1/3)

<ipython-input-7-dbb665e4361f>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  features = data[all_columns]


# Wrap the ANN around a custom sklearn classifier

In [ ]:
class ANNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, in_features=10, out_features=2, num_hidden=1, \
                 num_neurons=16, regularizer='l2',
                 reg_param=1e-4, batch_norm=False, epochs=100, \
                 early_stopping=True, verbose=1):
        self.in_features = in_features
        self.out_features = out_features
        self.num_hidden = num_hidden
        self.num_neurons = num_neurons
        self.regularizer = regularizer
        self.regularizer_func = regularizers.l2 if regularizer=='l2' else regularizers.l1
        self.reg_param = reg_param
        self.batch_norm = batch_norm
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.verbose = verbose

        # Build the ANN
        self.model = ANNClassifier.build_model(in_features, out_features, \
                                               self.num_hidden, \
                                               self.num_neurons, \
                                               self.regularizer_func, \
                                               self.reg_param, \
                                               self.batch_norm)

    @staticmethod
    def build_model(in_features, out_features, \
                    num_hidden, num_neurons, \
                    regularizer_func, reg_param, batch_norm):
        inputs = Input(shape=(in_features, ))
        outputs = inputs

        for i in range(num_hidden):
            outputs = Dense(num_neurons, kernel_regularizer=regularizer_func(reg_param), activation='relu')(outputs)
            if(batch_norm):
                outputs = BatchNormalization()(outputs)

        outputs = Dense(out_features, activation='softmax')(outputs)
        model = Model(inputs=inputs, outputs=outputs, name='ANNClassifier')

        return model

    def fit(self, X, Y):
        # Split into training and validating sets
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=1/3)

        # Specifies callbacks list
        callbacks = [
            ModelCheckpoint('model.weights.hdf5', save_best_only=True, verbose=self.verbose)
        ]

        # If early stopping is on, append early stopping callback to list
        if(self.early_stopping):
            callbacks.append(EarlyStopping(monitor='val_loss', patience=10, verbose=self.verbose))

        # Configure training process
        adam = Adam(lr=0.001, amsgrad=True)
        loss = SparseCategoricalCrossentropy()
        accuracy = SparseCategoricalAccuracy()

        # Compile the model then train
        self.model.compile(optimizer=adam, loss=loss, metrics=[accuracy])
        self.model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=self.epochs,
                       callbacks=callbacks, verbose=self.verbose)

    def predict(self, X):
        predictions = self.model.predict(X)
        predictions = np.argmax(predictions, axis=1)

        return predictions

# Train a sample ANN

In [ ]:
in_features = features.shape[1]
out_features = len(np.unique(targets))

ann = ANNClassifier(in_features, out_features, epochs=10, verbose=1)
ann.fit(X_train, Y_train)

Epoch 1/10
849/849 [==============================] - ETA: 0s - loss: 1.0627 - sparse_categorical_accuracy: 0.4730
Epoch 1: val_loss improved from inf to 0.98781, saving model to model.weights.hdf5
849/849 [==============================] - 6s 5ms/step - loss: 1.0627 - sparse_categorical_accuracy: 0.4730 - val_loss: 0.9878 - val_sparse_categorical_accuracy: 0.4942
Epoch 2/10
 29/849 [>.............................] - ETA: 2s - loss: 0.9732 - sparse_categorical_accuracy: 0.5011

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


849/849 [==============================] - ETA: 0s - loss: 0.9601 - sparse_categorical_accuracy: 0.5054
Epoch 2: val_loss improved from 0.98781 to 0.95168, saving model to model.weights.hdf5
849/849 [==============================] - 7s 8ms/step - loss: 0.9601 - sparse_categorical_accuracy: 0.5054 - val_loss: 0.9517 - val_sparse_categorical_accuracy: 0.4952
Epoch 3/10
833/849 [============================>.] - ETA: 0s - loss: 0.9295 - sparse_categorical_accuracy: 0.5095
Epoch 3: val_loss improved from 0.95168 to 0.92660, saving model to model.weights.hdf5
849/849 [==============================] - 3s 3ms/step - loss: 0.9296 - sparse_categorical_accuracy: 0.5090 - val_loss: 0.9266 - val_sparse_categorical_accuracy: 0.5008
Epoch 4/10
840/849 [============================>.] - ETA: 0s - loss: 0.9093 - sparse_categorical_accuracy: 0.5156
Epoch 4: val_loss improved from 0.92660 to 0.90890, saving model to model.weights.hdf5
849/849 [==============================] - 2s 3ms/step - loss: 0.90

# Fine-tuning the ANN

In [ ]:
param_grid = {
    'num_hidden' : [1, 2],
    'num_neurons' : [8, 16],
    'reg_param' : [2e-4, 2e-3]
}

clf = ANNClassifier(in_features=in_features, out_features=out_features, verbose=1, epochs=3)
gridsearchcv = GridSearchCV(clf, param_grid, cv=3, scoring='accuracy', return_train_score=True, verbose=3)
gridsearchcv.fit(X_train, Y_train)
best_params = gridsearchcv.best_params_

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
547/566 [===========================>..] - ETA: 0s - loss: 1.1971 - sparse_categorical_accuracy: 0.4233
Epoch 1: val_loss improved from inf to 1.03330, saving model to model.weights.hdf5
566/566 [==============================] - 3s 3ms/step - loss: 1.1904 - sparse_categorical_accuracy: 0.4261 - val_loss: 1.0333 - val_sparse_categorical_accuracy: 0.4773
Epoch 2/3
 76/566 [===>..........................] - ETA: 0s - loss: 1.0240 - sparse_categorical_accuracy: 0.4807

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


565/566 [============================>.] - ETA: 0s - loss: 0.9972 - sparse_categorical_accuracy: 0.4937
Epoch 2: val_loss improved from 1.03330 to 0.98863, saving model to model.weights.hdf5
566/566 [==============================] - 2s 4ms/step - loss: 0.9970 - sparse_categorical_accuracy: 0.4938 - val_loss: 0.9886 - val_sparse_categorical_accuracy: 0.4868
Epoch 3/3
552/566 [============================>.] - ETA: 0s - loss: 0.9619 - sparse_categorical_accuracy: 0.5055
Epoch 3: val_loss improved from 0.98863 to 0.96561, saving model to model.weights.hdf5
849/849 [==============================] - 2s 2ms/step


[CV 1/3] END num_hidden=1, num_neurons=8, reg_param=0.0002;, score=(train=0.505, test=0.500) total time=  12.4s
Epoch 1/3
559/566 [============================>.] - ETA: 0s - loss: 1.1699 - sparse_categorical_accuracy: 0.4355
Epoch 1: val_loss improved from inf to 1.01833, saving model to model.weights.hdf5
566/566 [==============================] - 3s 5ms/step - loss: 1.1680 - sparse_categorical_accuracy: 0.4362 - val_loss: 1.0183 - val_sparse_categorical_accuracy: 0.4959
Epoch 2/3
 42/566 [=>............................] - ETA: 1s - loss: 1.0205 - sparse_categorical_accuracy: 0.4814

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


552/566 [============================>.] - ETA: 0s - loss: 1.0114 - sparse_categorical_accuracy: 0.4900
Epoch 2: val_loss improved from 1.04491 to 0.99753, saving model to model.weights.hdf5
566/566 [==============================] - 2s 3ms/step - loss: 1.0109 - sparse_categorical_accuracy: 0.4899 - val_loss: 0.9975 - val_sparse_categorical_accuracy: 0.4847
Epoch 3/3
548/566 [============================>.] - ETA: 0s - loss: 0.9727 - sparse_categorical_accuracy: 0.4981
Epoch 3: val_loss improved from 0.99753 to 0.96973, saving model to model.weights.hdf5
849/849 [==============================] - 2s 2ms/step


[CV 3/3] END num_hidden=2, num_neurons=16, reg_param=0.002;, score=(train=0.502, test=0.502) total time=  12.6s
Epoch 1/3
846/849 [============================>.] - ETA: 0s - loss: 1.0898 - sparse_categorical_accuracy: 0.4608
Epoch 1: val_loss improved from inf to 0.98997, saving model to model.weights.hdf5
849/849 [==============================] - 5s 5ms/step - loss: 1.0894 - sparse_categorical_accuracy: 0.4608 - val_loss: 0.9900 - val_sparse_categorical_accuracy: 0.4922
Epoch 2/3
 71/849 [=>............................] - ETA: 1s - loss: 0.9870 - sparse_categorical_accuracy: 0.5035

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


836/849 [============================>.] - ETA: 0s - loss: 0.9808 - sparse_categorical_accuracy: 0.4950
Epoch 2: val_loss improved from 0.98997 to 0.95315, saving model to model.weights.hdf5
849/849 [==============================] - 3s 4ms/step - loss: 0.9805 - sparse_categorical_accuracy: 0.4949 - val_loss: 0.9531 - val_sparse_categorical_accuracy: 0.5066
Epoch 3/3
841/849 [============================>.] - ETA: 0s - loss: 0.9553 - sparse_categorical_accuracy: 0.5025
Epoch 3: val_loss improved from 0.95315 to 0.93358, saving model to model.weights.hdf5
849/849 [==============================] - 6s 7ms/step - loss: 0.9551 - sparse_categorical_accuracy: 0.5025 - val_loss: 0.9336 - val_sparse_categorical_accuracy: 0.5081


In [ ]:
print('Best model parameters : ', best_params)

Best model parameters :  {'num_hidden': 1, 'num_neurons': 16, 'reg_param': 0.0002}


# Evaluate the best model

In [ ]:
# Build an ANN based on the best params set
clf =  ANNClassifier(in_features=in_features, out_features=out_features, verbose=1, epochs=5, **best_params)
clf.fit(X_train, Y_train)

Epoch 1/5
842/849 [============================>.] - ETA: 0s - loss: 1.0635 - sparse_categorical_accuracy: 0.4963
Epoch 1: val_loss improved from inf to 1.00425, saving model to model.weights.hdf5
849/849 [==============================] - 3s 3ms/step - loss: 1.0625 - sparse_categorical_accuracy: 0.4970 - val_loss: 1.0043 - val_sparse_categorical_accuracy: 0.4986
Epoch 2/5
 74/849 [=>............................] - ETA: 1s - loss: 0.9949 - sparse_categorical_accuracy: 0.5084

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


831/849 [============================>.] - ETA: 0s - loss: 0.9708 - sparse_categorical_accuracy: 0.5140
Epoch 2: val_loss improved from 1.00425 to 0.96728, saving model to model.weights.hdf5
849/849 [==============================] - 2s 3ms/step - loss: 0.9703 - sparse_categorical_accuracy: 0.5138 - val_loss: 0.9673 - val_sparse_categorical_accuracy: 0.4980
Epoch 3/5
844/849 [============================>.] - ETA: 0s - loss: 0.9431 - sparse_categorical_accuracy: 0.5130
Epoch 3: val_loss improved from 0.96728 to 0.94524, saving model to model.weights.hdf5
849/849 [==============================] - 2s 3ms/step - loss: 0.9431 - sparse_categorical_accuracy: 0.5129 - val_loss: 0.9452 - val_sparse_categorical_accuracy: 0.4972
Epoch 4/5
847/849 [============================>.] - ETA: 0s - loss: 0.9261 - sparse_categorical_accuracy: 0.5119
Epoch 4: val_loss improved from 0.94524 to 0.93048, saving model to model.weights.hdf5
849/849 [==============================] - 3s 3ms/step - loss: 0.9260

In [ ]:
# Make predictions on the testing set
predictions = clf.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)

print('Accuracy on testing set : ', accuracy)

637/637 [==============================] - 1s 1ms/step
Accuracy on testing set :  0.5080316353097215
